In [8]:
from entropy_compressor import ZCompressor
from quantizator import Quantizator
from wavelet_compressor import WaveletCompressor

import os
import shutil
import numpy as np
from typing import Dict, List
from pathlib import Path
from pprint import pprint

COMPRESS_PATH = 'compressed_low'
DECOMPRESS_PATH = 'decompressed'
IMAGE_NAME = 'low.png'
MESH_KEYS = ['faces', 'neighbour', 'owner', 'points']
PTURHO_KEYS = ['p', 'T', 'U', 'rho']

class Decompressor:
    def __init__(self):
        if not os.path.exists(DECOMPRESS_PATH):
            os.mkdir(DECOMPRESS_PATH)
        self.entropy_compressor = ZCompressor()
        self.quantizator = Quantizator()
        self.wavelet_compressor = WaveletCompressor()
    
    def total_decompress(self, compressed_data: str, q_step: np.float64, min_val: np.float64) -> np.ndarray:
        huffman_decoded = self.entropy_compressor.decompress(bytes.fromhex(compressed_data))
        dequantized = self.quantizator.dequantize(huffman_decoded, q_step, min_val)
        decompressed = self.wavelet_compressor.inverse_wavelet_transform(dequantized)

        return decompressed

    def load_to_file(self, data: List[np.ndarray | np.float64], file_path: str, header_data: dict[str]) -> None:
        huffman_compressed, q_step, min_val = data
        decompressed = self.total_decompress(huffman_compressed, q_step, min_val)
        
    def read_mesh_file(self, file_path: str) -> Dict[str, List[np.ndarray | np.float64]]:
        data = {}
        for key in MESH_KEYS:
            data[key] = None
        i = 0
        with open(file_path, mode='r') as f:
            while True:
                # Read the first line
                quantilization_keys = f.readline().strip()
                # Read the second line
                huffman_compressed = f.readline().strip()

                # Check if either line is empty, indicating the end of the file
                if not quantilization_keys or not huffman_compressed:
                    break
                
                q_step, min_val = map(float, quantilization_keys.split())
                data[MESH_KEYS[i]] = [huffman_compressed, q_step, min_val]
                i += 1
        return data
    
    def read_pturho_file(self, file_path: str) -> Dict[str, List[np.ndarray | np.float64]]:
        data = {}
        i = 0
        with open(file_path, mode='r') as f:
            times = f.readline().strip().split(';')
            for key in times:
                data[key] = None
            while True:
                quantilization_keys_p = f.readline().strip()
                huffman_compressed_p = f.readline().strip()

                quantilization_keys_t = f.readline().strip()
                huffman_compressed_t = f.readline().strip()

                quantilization_keys_u = f.readline().strip()
                huffman_compressed_u = f.readline().strip()

                quantilization_keys_rho = f.readline().strip()
                huffman_compressed_rho = f.readline().strip()

                # Check if either line is empty, indicating the end of the file
                if not quantilization_keys_p or not huffman_compressed_p:
                    break
                
                q_step_p, min_val_p = map(float, quantilization_keys_p.split())
                q_step_t, min_val_t = map(float, quantilization_keys_t.split())
                q_step_u, min_val_u = map(float, quantilization_keys_u.split())
                q_step_rho, min_val_rho = map(float, quantilization_keys_rho.split())
                data[times[i]] = [
                    [huffman_compressed_p, q_step_p, min_val_p],
                    [huffman_compressed_t, q_step_t, min_val_t],
                    [huffman_compressed_u, q_step_u, min_val_u],
                    [huffman_compressed_rho, q_step_rho, min_val_rho],
                ]
                i += 1
        return data

    def copy_back_not_compressed_piece(self, file_path: str, destination_path: str) -> None:
        # shutil
        pass

In [9]:
decompressor = Decompressor()
mesh_data = decompressor.read_mesh_file('compressed_low/mesh')

In [10]:
faces, qstep, minval = mesh_data['faces']
print(faces, qstep, minval)
decompressed_faces = decompressor.total_decompress(faces, qstep, minval)

b'x\x9cU\x95mL\x97U\x18\xc6OQ\xac\t\xab\xb6\xe4\x83\xce\xe5\xd3\xd0\xda\xdf9Wj\xbc8\x8bck4\x9bP1\x9d\xcc\xc2\x1eEKg\x99\xab\xe8\xc5F;\x8d\x19\x96_X\x0e\x97\xd8\xe4\xc1\xb1\xfdq:\x87/I\x81\xe4#\xe0\x0b"\xa1(9Q\xf2\x91\x14\x14\x14u2u\xb6\xd9\x0b\xd7\xaf\x0f\xfd\xbf\\\xbb\xaf\xeb\xbe\xefs\xce\xfd\x9cs\xfd\x8d\xf9\xe7\x97Qg\xff\x05\xb3f\xbf\xd0\x1e\x15\xae\xfeU8\xd4(\x9cph\x04]g\xab\xe2/\x0f\x0b\x1f\xfbY\xfc\xb8\x03\x8a\xabT\xef\x06\xdb\x15\x174\t\xbd6\xf1/\x80\x9f\x1f\x14\xffv(\\\xab\xd8\x85\xd4\xb5u\x08\xb74\x0b;\xe8[\xac}\xb9\xd81a\xa6t\xb7\xf8\x880\xe7\xf8\x08\x86y\'\x14\x97\xd1w\x03\xeb\x0eI7\x0f\xa17\xea<\xd6*6\xaf\xc1w\xab\xbf\tuNw\x99\xf5\xca:\xd5\xbf\xfc\xa4\xe2\xd7\xe9[\xa8\xba\xb0\x1f\xfd}\xfa\xccfN\xb3\x98k\xbb\xea\xcc\xf4\xd3\xc2r\xd5\x9b\xf1:\xb7=~F\xf1\xd4H\xf1g\xe7G0\xda\xd5+~\xc5%\xf5M\x06?\x95\xee/\x95\x1e\xd6_T\xde\x17B?S}\xcc\x00z\x8a\xeal#y\x0b/\x8c\xa0\xb7\x96x\xb9\xf4\xe8\xe6\x1f\xca\x8f\x9f\xa5\x8e>w\x94\xe7\xbd\xd8\'\xbeO|\xf4\xbd\xf2\xdd]\xd5\xfb\xbb\xe8S\xaa\xfe.

error: Error -3 while decompressing data: incorrect header check